In [ ]:
# # Use a pipeline as a high-level helper
# Load model directly
# 使用国内镜像网站
import json
import pdfplumber
import jieba
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
# 加载模型
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("BAAI/bge-small-zh-v1.5")
#model = SentenceTransformer("Alibaba-NLP/gte-Qwen2-7B-instruct")#no
#model = SentenceTransformer("Alibaba-NLP/gte-Qwen1.5-7B-instruct")#no
#model = SentenceTransformer("BAAI/bge-large-en-v1.5")#no


#读取问答数据集
questions = json.load(open("questions.json",encoding='utf-8'))
print('questions[0]:',questions[0])
pdf = pdfplumber.open("初赛训练数据集.pdf")
len(pdf.pages) # 页数
pdf.pages[0].extract_text() # 读取第一页内容
# 读取所有页内容
pdf_content = []
for page_idx in range(len(pdf.pages)):
    pdf_content.append({
        'page': 'page_' + str(page_idx + 1),
        'content': pdf.pages[page_idx].extract_text()
    })
# 对问题和文档内容进行分词
question_sentences = [x['question'] for x in questions]
pdf_content_sentences = [x['content'] for x in pdf_content]
# 对问题和pdf内容做嵌入，也就是文本编码
question_embeddings = model.encode(question_sentences, normalize_embeddings=True)
pdf_embeddings = model.encode(pdf_content_sentences, normalize_embeddings=True)
# 进行相似度打分和结果排序找到最相关的文档
for query_idx, feat in enumerate(question_embeddings):
    score = feat @ pdf_embeddings.T
    max_score_page_idx = score.argsort()[-1] + 1
    questions[query_idx]['reference'] = 'page_' + str(max_score_page_idx)

with open('submit_bge.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

questions[0]: {'question': '“前排座椅通风”的相关内容在第几页？', 'answer': '', 'reference': ''}
